STEP #0: Import libraries

In [53]:
import pandas as pd
import sqlite3
import numpy as np

STEP #1: Connect to database

In [ ]:
conn = sqlite3.connect('../data/ecommerce.db')

df = pd.read_sql('SELECT * FROM transaction_raw', conn)

print(f"Starting with {len(df)} rows")

STEP #2: Clean data

In [ ]:
df_clean = df.copy()

df_clean = df_clean[~df_clean['InvoiceNo'].str.startswith('C')]  # Drop cancelled orders in InvoiceNo col

df_clean = df_clean[df_clean['Quantity'] > 0]                    # Keep Quantity > 0

df_clean = df_clean[df_clean['UnitPrice'] > 0]                   # Keep UnitPrice > 0

df_clean = df_clean[df_clean['CustomerID'].notna()]              # Remove rows with missing CustomerID

df_clean = df_clean.drop_duplicates()                            # Remove duplicate rows

display(df_clean)
print(f"Original rows: {len(df)}")
print(f"Rows after cleaning: {len(df_clean)}")
print(f"Dropped rows: {len(df) - len(df_clean)}")

STEP #3: Calculate new columns

In [ ]:
df_clean = df_clean.copy()
df_clean['TotalPrice'] = df_clean['UnitPrice'] * df['Quantity']
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'], format='mixed', errors='coerce')
df_clean['Year'] = df_clean['InvoiceDate'].dt.year
df_clean['Month'] = df_clean['InvoiceDate'].dt.month
df_clean['YearMonth'] = df_clean['InvoiceDate'].dt.strftime('%Y-%m')

display(df_clean.head(10))

STEP #4: Write df_clean to sqlite and verify

In [ ]:
df_clean.to_sql('transactions_clean', conn, if_exists='replace', index=False)

query_check = """
SELECT *
FROM transactions_clean
LIMIT 10;
"""
query_tot_rev = """
SELECT
    YearMonth, 
    SUM(TotalPrice) AS Total_revenue
FROM transactions_clean
GROUP BY YearMonth
ORDER BY YearMonth
"""

display(pd.read_sql(query_check, conn))
display(pd.read_sql(query_tot_rev, conn))